In [ ]:
import torch
from leap import LeapForCausalLM, LeapConfig
from transformers import TrainingArguments, Trainer, default_data_collator
from datasets import load_dataset

from itertools import chain
import logging
logging.disable(logging.INFO)

In [ ]:
# t5 tokenzier, warning is nothing to worry about since we will group the texts
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
raw_datasets = load_dataset("wikitext", "wikitext-2-v1")
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]
block_size = 2048

def tokenize_function(examples):
    output = tokenizer(examples[text_column_name])
    return output

def group_texts(examples):
    # concatenate text
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # drop last block
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size

    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=1,
            remove_columns=column_names,
        )

lm_dataset = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=1,
    desc=f"Grouping texts in chunks of {block_size}",
)

lm_dataset.set_format('pt')

In [ ]:
# hyperparameters
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "epoch",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    report_to = "none",
    learning_rate = 5e-4, 
    num_train_epochs = 10,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    max_grad_norm = 1,
    fp16 = True
)

In [ ]:
def run_leap(window_sizes):
    config = LeapConfig(
        hidden_size = 128,
        vocab_size = len(tokenizer),
        n_positions = block_size,
        n_heads = 4,
        n_layer = len(window_sizes),
        use_local_att = True,
        window_sizes = window_sizes, # will be set automatically
        hidden_dropout_prob = .1,
        rescale = 10
    )
    print(config.window_sizes)
    window_model = LeapForCausalLM(config)
    
    window_trainer = Trainer(
        model=window_model,
        args=training_args,
        data_collator=default_data_collator,
        train_dataset=lm_dataset["train"],
        eval_dataset=lm_dataset["validation"]
    )

    window_trainer.train()
    
    # free gpu memory
    del window_trainer
    window_model.cpu()
    torch.cuda.empty_cache()

# Simple experiments to find a heuristic for window values

Having the first and last layer be global attention is for sure. The first global attention layer should establish "context" like if there's task metadata at the start of the sequence. The last global attention layer to accumulate  all information. 

# 1. Let's start with a baseline

The first and last layers are global attention (as stated before) and of course we need local attention so let's try a window size of 4.

In [ ]:
run_leap([2048, 4, 2048])

[2048, 4, 2048]


Epoch,Training Loss,Validation Loss
1,6.403400,5.614212
2,5.718500,5.347041
3,5.473900,5.140350
4,5.281300,5.001758
5,5.145200,4.916826
6,5.044400,4.843692
7,4.970400,4.792968
8,4.915800,4.760973
9,4.877600,4.742640
10,4.855600,4.734987


### 2. Now try a local window size of 8 to compare

In [ ]:
run_leap([2048, 8, 2048])

[2048, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.417200,5.626072
2,5.730600,5.369969
3,5.491900,5.174758
4,5.310600,5.040751
5,5.180400,4.953936
6,5.083100,4.892673
7,5.010500,4.835805
8,4.957700,4.805156
9,4.920500,4.786395
10,4.898000,4.778767


## 3. Seems slightly worse, let's move on. Try adding a second local attention layer vs adding another global attention layer

In [ ]:
run_leap([2048, 4, 4, 2048])

[2048, 4, 4, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.408400,5.614400
2,5.715400,5.354980
3,5.476300,5.146724
4,5.277700,4.997527
5,5.132900,4.891938
6,5.029600,4.823465
7,4.951300,4.765365
8,4.894500,4.735488
9,4.855300,4.713768
10,4.831800,4.704624


In [ ]:
run_leap([2048, 4, 2048, 2048])

[2048, 4, 2048, 2048]


Epoch,Training Loss,Validation Loss
1,6.397600,5.615664
2,5.720400,5.347426
3,5.468500,5.142619
4,5.278800,5.005457
5,5.145600,4.910377
6,5.047000,4.841533
7,4.972100,4.789617
8,4.916700,4.759144
9,4.877400,4.736964
10,4.854500,4.729196


## 4. How about also trying 8

In [ ]:
run_leap([2048, 4, 8, 2048])

[2048, 4, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.407400,5.610317
2,5.710700,5.349531
3,5.468800,5.135484
4,5.268200,4.987660
5,5.124800,4.884447
6,5.021300,4.814922
7,4.942300,4.753984
8,4.885200,4.722650
9,4.845500,4.700609
10,4.821600,4.691215


## 5. Okay, the 4 and 8 local windows are the best so far. Let's try another local window of double the size of the previous local window comparing against just adding another 8 layer

In [ ]:
run_leap([2048, 4, 8, 16, 2048])

[2048, 4, 8, 16, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.413400,5.598911
2,5.693700,5.320351
3,5.438600,5.114291
4,5.245200,4.965509
5,5.103500,4.867014
6,4.999400,4.791622
7,4.920800,4.734686
8,4.863700,4.698231
9,4.822700,4.677152
10,4.800600,4.668426


In [ ]:
run_leap([2048, 4, 8, 8, 2048])

[2048, 4, 8, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.415500,5.602804
2,5.698700,5.326443
3,5.445000,5.120928
4,5.251900,4.972217
5,5.110500,4.872621
6,5.006700,4.796437
7,4.927900,4.740082
8,4.870900,4.704421
9,4.830100,4.683311
10,4.808000,4.674782


## 6. Adding 16 is best, let's continue the pattern of adding a local window of double the size. Let's compare against adding a 4 instead, and try also another global layer

In [ ]:
run_leap([2048, 4, 8, 16, 32, 2048])

[2048, 4, 8, 16, 32, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.368600,5.593536
2,5.676700,5.310952
3,5.425300,5.093761
4,5.230900,4.957909
5,5.091400,4.851718
6,4.986400,4.780315
7,4.907500,4.719919
8,4.849100,4.683742
9,4.809200,4.665123
10,4.784500,4.655414


In [ ]:
run_leap([2048, 4, 8, 16, 4, 2048])

[2048, 4, 8, 16, 4, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.370000,5.593909
2,5.677300,5.309531
3,5.423700,5.092208
4,5.228600,4.955739
5,5.089800,4.849909
6,4.985100,4.780025
7,4.906500,4.718997
8,4.847800,4.682447
9,4.807900,4.663561
10,4.782900,4.653922


In [ ]:
run_leap([2048, 4, 8, 16, 2048, 2048])

[2048, 4, 8, 16, 2048, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.370100,5.590558
2,5.675600,5.310473
3,5.423300,5.092281
4,5.228500,4.954886
5,5.089400,4.850015
6,4.984800,4.778233
7,4.906700,4.720803
8,4.849100,4.686124
9,4.809500,4.666592
10,4.784900,4.657202


## 7. They all seem to work well, though restarting back to 4 seems to work best. Let's continue with adding and 8 vs adding a global layer BEFORE the 4 (the idea being to alternate between global and local attention)

In [ ]:
run_leap([2048, 4, 8, 16, 4, 8, 2048])

[2048, 4, 8, 16, 4, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.357700,5.587782
2,5.673200,5.309240
3,5.413900,5.093601
4,5.214500,4.946199
5,5.075600,4.841637
6,4.973500,4.767156
7,4.895100,4.711235
8,4.837000,4.674083
9,4.796600,4.652811
10,4.772200,4.644971


In [ ]:
run_leap([2048, 4, 8, 16, 2048, 4, 2048])

[2048, 4, 8, 16, 2048, 4, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.356200,5.586043
2,5.671900,5.305367
3,5.408200,5.082562
4,5.207600,4.938675
5,5.068900,4.835489
6,4.966600,4.762750
7,4.888800,4.705896
8,4.830600,4.670035
9,4.790600,4.648480
10,4.766400,4.639331


## 8. So alternation might work! If that's true let's try getting rid of the first global attention layer (and using the alternating strategy), vs leaving it and not alternating

In [ ]:
run_leap([4, 8, 16, 2048, 4, 8, 16, 2048])

[4, 8, 16, 2048, 4, 8, 16, 2048]


Epoch,Training Loss,Validation Loss
1,6.447600,5.701015
2,5.798900,5.437718
3,5.554800,5.219329
4,5.360300,5.061584
5,5.210400,4.942722
6,5.094500,4.855447
7,5.005600,4.793694
8,4.941300,4.749907
9,4.897200,4.725306
10,4.871700,4.716165


In [ ]:
run_leap([2048, 4, 8, 16, 4, 8, 16, 2048])

[2048, 4, 8, 16, 4, 8, 16, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.379700,5.578846
2,5.665700,5.309939
3,5.409700,5.077781
4,5.203500,4.924696
5,5.057800,4.821679
6,4.950900,4.743570
7,4.870600,4.690719
8,4.812900,4.654828
9,4.771900,4.633359
10,4.747100,4.624532


## 9. Okay so looks like there should be global attention up front (good to clarify that at least), let's back up and just try the alternating strategy with global attention up front (comparing non alternating strategy directly above)

In [ ]:
run_leap([2048, 4, 8, 16, 2048, 4, 8, 2048])

[2048, 4, 8, 16, 2048, 4, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.379300,5.576530
2,5.666200,5.304617
3,5.407900,5.078589
4,5.200300,4.926067
5,5.053500,4.822440
6,4.946000,4.742070
7,4.865100,4.686777
8,4.806900,4.651878
9,4.765700,4.628971
10,4.740800,4.619902


## 10. That seems to work well, let's compare that to maybe trying faster cycle rates (compared to baseline 4, 8, 16)

In [ ]:
run_leap([2048, 4, 16, 2048, 4, 16, 2048, 4, 2048])

[2048, 4, 16, 2048, 4, 16, 2048, 4, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.337400,5.570338
2,5.654300,5.292899
3,5.396700,5.068539
4,5.196800,4.919131
5,5.054200,4.819403
6,4.949100,4.747835
7,4.869400,4.690602
8,4.810800,4.650536
9,4.768500,4.629301
10,4.744200,4.620091


In [ ]:
run_leap([2048, 4, 8, 2048, 4, 8, 2048, 4, 2048])

[2048, 4, 8, 2048, 4, 8, 2048, 4, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.341100,5.579933
2,5.662300,5.301711
3,5.406600,5.078926
4,5.203200,4.926170
5,5.061500,4.829375
6,4.956000,4.757246
7,4.877600,4.698025
8,4.818400,4.658959
9,4.776700,4.636538
10,4.752400,4.627797


In [ ]:
run_leap([2048, 8, 16, 2048, 8, 16, 2048, 8, 2048])

[2048, 8, 16, 2048, 8, 16, 2048, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.341200,5.585158
2,5.672700,5.320930
3,5.424400,5.105385
4,5.231100,4.962701
5,5.094200,4.868821
6,4.993000,4.796904
7,4.917200,4.742193
8,4.860800,4.706789
9,4.821200,4.685561
10,4.797400,4.676823


In [ ]:
run_leap([2048, 4, 8, 16, 2048, 4, 8, 16, 2048])

[2048, 4, 8, 16, 2048, 4, 8, 16, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.337000,5.573453
2,5.651800,5.286858
3,5.393000,5.064430
4,5.192500,4.914081
5,5.049900,4.814898
6,4.944500,4.743355
7,4.864900,4.685818
8,4.805700,4.646133
9,4.764000,4.622876
10,4.739900,4.614115


## 11. Looks like 4, 8, 16 is best. Though, what if the last layer wasn't global?

In [ ]:
run_leap([2048, 4, 8, 16, 2048, 4, 8, 16])

[2048, 4, 8, 16, 2048, 4, 8, 16]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.378000,5.577907
2,5.665000,5.303333
3,5.405800,5.074307
4,5.198600,4.922642
5,5.052600,4.819136
6,4.944600,4.740305
7,4.863700,4.685725
8,4.805600,4.650772
9,4.764200,4.627913
10,4.739400,4.618880


In [ ]:
run_leap([2048, 4, 8, 16, 2048, 4, 8, 2048])

[2048, 4, 8, 16, 2048, 4, 8, 2048]


C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,6.379300,5.576530
2,5.666200,5.304617
3,5.407900,5.078589
4,5.200300,4.926067
5,5.053500,4.822440
6,4.946000,4.742070
7,4.865100,4.686777
8,4.806900,4.651878
9,4.765700,4.628971
10,4.740800,4.619902


# Conclusion for now: first layer should be global attention, layers of sizes 4, then 8, then 16, then global attention, and repeat.